In [1]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import entropy
import torch.nn.functional as F
from scipy.signal import savgol_filter
from statsmodels.nonparametric.smoothers_lowess import lowess

In [2]:
def process_signal(signal, gain, offset, dead, flat, dark, linear_corr):
    signal = signal * gain + offset
    flat = np.ma.masked_where(dead, flat)
    dark = np.ma.masked_where(dead, dark)
    flat = np.tile(flat, (signal.shape[0], 1, 1))
    dark = np.tile(dark, (signal.shape[0], 1, 1))
    dead = np.tile(dead, (signal.shape[0], 1, 1))
    
    signal = np.ma.masked_where(dead, signal)
    signal = (signal - dark) / (flat - dark)
#     linear_corr = np.flip(linear_corr, axis=0)
#     for x in range(signal.shape[1]):
#         for y in range(signal.shape[2]):
#             poli = np.poly1d(linear_corr[:, x, y])
#             signal[:, x, y] = poli(signal[:, x, y])
    signal = signal[1::2, :, :] - signal[::2, :, :]    
    return signal

def bin_obs(cds_signal, binning):
    cds_transposed = cds_signal.transpose(1, 2, 0)
    binned_shape = (cds_transposed.shape[0], cds_transposed.shape[1], cds_transposed.shape[2] // binning)
    cds_binned = np.zeros(binned_shape)
    for i in range(binned_shape[2]):
        cds_binned[:, :, i] = np.sum(cds_transposed[:, :, i*binning:(i+1)*binning], axis=2)
    return cds_binned.transpose(2, 0, 1)


In [3]:
def smooth_data(data, window_size):
    return savgol_filter(data, window_size, 3)

def optimize_breakpoint(data, initial_breakpoint, window_size, buffer_size, smooth_window):
    best_breakpoint = initial_breakpoint
    best_score = float("-inf")
    midpoint = len(data) // 2
    smoothed_data = smooth_data(data, smooth_window)
#     smoothed_data=data
    for i in range(-window_size, window_size):
        new_breakpoint = initial_breakpoint + i
        if new_breakpoint > buffer_size and new_breakpoint < midpoint - buffer_size:
            region1 = data[: new_breakpoint - buffer_size]
            region2 = data[
                new_breakpoint
                + buffer_size : 2 * midpoint
                - new_breakpoint
                - buffer_size
            ]
            region3 = data[2 * midpoint - new_breakpoint + buffer_size :]

            breakpoint_region1 = smoothed_data[new_breakpoint - buffer_size: new_breakpoint + buffer_size]
            breakpoint_region2 = smoothed_data[new_breakpoint - buffer_size: new_breakpoint + buffer_size]

            mean_diff = abs(np.mean(region1) - np.mean(region2)) + abs(
                np.mean(region2) - np.mean(region3)
            )
            var_sum = np.var(region1) + np.var(region2) + np.var(region3)
            range_at_breakpoint1 = (np.max(breakpoint_region1) - np.min(breakpoint_region1))
            range_at_breakpoint2 = (np.max(breakpoint_region2) - np.min(breakpoint_region2))

            mean_range_at_breakpoint = (range_at_breakpoint1 + range_at_breakpoint2) / 2

            score = mean_diff - 0.5 * var_sum + mean_range_at_breakpoint

            if score > best_score:
                best_score = score
                best_breakpoint = new_breakpoint

                
    return best_breakpoint

In [4]:
class ResidualModel(nn.Module):
    def __init__(self):
        super(ResidualModel, self).__init__()
        self.fc0 = nn.Linear(24, 48)
        self.fc1 = nn.Linear(48, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 256)
        self.fc4 = nn.Linear(256, 283)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc0(x))
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [5]:
class CNN2D_AIRS(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CNN2D_AIRS, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
#         x = self.pool(F.relu(self.bn1(self.conv1(x))))
#         x = self.pool(F.relu(self.bn2(self.conv2(x))))

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        return x

class CNN2D_FGS(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CNN2D_FGS, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
#         x = self.pool(F.relu(self.bn1(self.conv1(x))))
#         x = self.pool(F.relu(self.bn2(self.conv2(x))))

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        return x

class Model(nn.Module):
    def __init__(self, airs_frames, fgs_frames):
        super(Model, self).__init__()
        self.cnn_airs = CNN2D_AIRS(1, 16)
        self.cnn_fgs = CNN2D_FGS(1, 16)
        
        self.lstm_airs = nn.LSTM(16 * 8 * 89, 128, batch_first=True)
        self.lstm_fgs = nn.LSTM(16 * 8 * 8, 128, batch_first=True)
        
        self.bn_airs = nn.BatchNorm1d(128)
        self.bn_fgs = nn.BatchNorm1d(128)
        
        self.fc_light_curve_airs = nn.Sequential(
            nn.Linear(airs_frames, 64),
            nn.ReLU()
#             nn.BatchNorm1d(64),

        )
        
        self.fc_light_curve_fgs = nn.Sequential(
            nn.Linear(fgs_frames, 64),
            nn.ReLU()
#             nn.BatchNorm1d(64),

        )
        
        self.fc_combined = nn.Sequential(            
            nn.Linear(128 + 128 + 64 + 64, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
#             nn.Dropout(0.15),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
#             nn.Dropout(0.15),
            nn.Linear(256, 283)
        )

    def forward(self, airs_ch0, fgs1, light_curve_airs, light_curve_fgs):
        batch_size, frames, _, _, _ = airs_ch0.shape
        
        airs_features = self.cnn_airs(airs_ch0.view(-1, 1, 32, 356))
        airs_features = airs_features.view(batch_size, frames, -1)
        _, (airs_hidden, _) = self.lstm_airs(airs_features)
#         airs_hidden = self.bn_airs(airs_hidden.squeeze(0))
        airs_hidden = airs_hidden.squeeze(0)
        
        
        fgs_features = self.cnn_fgs(fgs1.view(-1, 1, 32, 32))
        fgs_features = fgs_features.view(batch_size, frames, -1)
        _, (fgs_hidden, _) = self.lstm_fgs(fgs_features)
#         fgs_hidden = self.bn_fgs(fgs_hidden.squeeze(0))
        fgs_hidden = fgs_hidden.squeeze(0)
        
        light_curve_airs_features = self.fc_light_curve_airs(light_curve_airs)
        light_curve_fgs_features = self.fc_light_curve_fgs(light_curve_fgs)
        
        combined_features = torch.cat((airs_hidden, fgs_hidden, light_curve_airs_features, light_curve_fgs_features), dim=1)
        
        output = self.fc_combined(combined_features)
        return output


In [6]:
        
def vector(data, start, end , start1 , end1):
    
    region1= data[:start]  
    
    region2=data[start:end]
    region3=data[end:start1]
    region4=data[start1:end1]

    region5=data[end1:]

    uncovered = (  np.mean(region1) + np.mean(region5)  )/2
    
    reduction1= ( uncovered - np.mean(region2)    ) / uncovered
    reduction2= ( uncovered - np.mean(region3)   )  / uncovered
    reduction3= ( uncovered - np.mean(region4)   )  / uncovered
        
    
    slope=(data[end]-data[start])/(end-start)
    slope1=(data[end1]-data[start1])/(end1-start1)
    
    nr = np.mean(data)    / np.std(data)
    nr1= np.mean(region1) / np.std(region1)
    nr2= np.mean(region2) / np.std(region2)
    nr3= np.mean(region3) / np.std(region3)
    nr4= np.mean(region4) / np.std(region4)
    nr5= np.mean(region5) / np.std(region5)
   
    skewness= skew(data)

    input_vector = np.array([ slope, slope1 , reduction1, reduction2, reduction3 ,nr,nr1,nr2,nr3,nr4,nr5 ,skewness] )

    return input_vector


def final_vector(airs_arr , fgs_arr):

    airs_arr=(airs_arr-np.min(airs_arr))/(np.max(airs_arr)-np.min(airs_arr))
    fgs_arr=(fgs_arr-np.min(fgs_arr))/(np.max(fgs_arr)-np.min(fgs_arr))
    
    initial_breakpoint=850
    buffer_size=80 
    smooth_window=200
    window_size=300
    airsbp = optimize_breakpoint(airs_arr,initial_breakpoint,window_size=window_size,buffer_size=buffer_size,smooth_window=smooth_window)
    fgsbp = optimize_breakpoint(fgs_arr,initial_breakpoint,window_size=window_size,buffer_size=buffer_size,smooth_window=250)
    midpoint1 = len(airs_arr) // 2
    bp1 = [airsbp, 2 * midpoint1 - airsbp]
    airs_start   =  bp1[0] - buffer_size
    airs_end     =  bp1[0] + buffer_size
    airs_start1  =  bp1[1] - buffer_size
    airs_end1    =  bp1[1] + buffer_size
    
    midpoint2 = len(fgs_arr) // 2
    bp2 = [fgsbp, 2 * midpoint2 - fgsbp]
    fgs_start  =    bp2[0] - buffer_size
    fgs_end    =    bp2[0] + buffer_size
    fgs_start1 =    bp2[1] - buffer_size
    fgs_end1   =    bp2[1] + buffer_size
    airs_vector=  vector( airs_arr,  airs_start ,  airs_end , airs_start1 , airs_end1 )
    fgs_vector =  vector( fgs_arr, fgs_start  ,   fgs_end , fgs_start1  , fgs_end1 )        
    
    f_vector=  np.concatenate((airs_vector , fgs_vector))
    
    return f_vector 

In [7]:
def planetnumber(filename):
    return int(filename.split('_')[0])

class PlanetDataset(Dataset):
    def __init__(self, mode):
        self.mode = mode
        self.adc_info = pd.read_csv(f'/kaggle/input/ariel-data-challenge-2024/{mode}_adc_info.csv')
        self.files = sorted(os.listdir(f"/kaggle/input/ariel-data-challenge-2024/{mode}"), key=planetnumber)
            
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        planet = int(self.files[idx])
        selected_adc_info = self.adc_info.loc[self.adc_info['planet_id'] == planet]
        
        # Load and process AIRS data
        airs_dark = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{self.mode}/{planet}/AIRS-CH0_calibration/dark.parquet').values.reshape(32, 356)
        airs_dead = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{self.mode}/{planet}/AIRS-CH0_calibration/dead.parquet').values.reshape(32, 356)
        airs_flat = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{self.mode}/{planet}/AIRS-CH0_calibration/flat.parquet').values.reshape(32, 356)
        airs_linear_corr = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{self.mode}/{planet}/AIRS-CH0_calibration/linear_corr.parquet').values.reshape(6, 32, 356)
        airs_ch0 = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{self.mode}/{planet}/AIRS-CH0_signal.parquet').values
        airs_ch0_gain = selected_adc_info['AIRS-CH0_adc_gain'].values[0]
        airs_ch0_offset = selected_adc_info['AIRS-CH0_adc_offset'].values[0]
        airs_ch0 = airs_ch0.reshape(11250, 32, 356)
        
        # Load and process FGS data
        fgs_dark = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{self.mode}/{planet}/FGS1_calibration/dark.parquet').values.reshape(32, 32)
        fgs_dead = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{self.mode}/{planet}/FGS1_calibration/dead.parquet').values.reshape(32, 32)
        fgs_flat = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{self.mode}/{planet}/FGS1_calibration/flat.parquet').values.reshape(32, 32)
        fgs_linear_corr = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{self.mode}/{planet}/FGS1_calibration/linear_corr.parquet').values.reshape(6, 32, 32)
        fgs1 = pd.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/{self.mode}/{planet}/FGS1_signal.parquet').values
        fgs1_gain = selected_adc_info['FGS1_adc_gain'].values[0]
        fgs1_offset = selected_adc_info['FGS1_adc_offset'].values[0]
        fgs1 = fgs1.reshape(135000, 32, 32)
        
        # Process signals
        airs_ch0_processed = process_signal(airs_ch0, airs_ch0_gain, airs_ch0_offset, airs_dead, airs_flat, airs_dark, airs_linear_corr)
        fgs1_processed = process_signal(fgs1, fgs1_gain, fgs1_offset, fgs_dead, fgs_flat, fgs_dark, fgs_linear_corr)
        
        # Bin observations
        airs_frames = bin_obs(airs_ch0_processed, binning=5)
        fgs_frames = bin_obs(fgs1_processed, binning=60)
        
        # Convert to PyTorch tensors and normalize
        airs_1d = torch.tensor(np.sum(airs_frames, axis=(1,2))).float()
        fgs_1d = torch.tensor(np.sum(fgs_frames, axis=(1,2))).float()
        
        airs_1d = (airs_1d - airs_1d.min()) / (airs_1d.max() - airs_1d.min())
        fgs_1d = (fgs_1d - fgs_1d.min()) / (fgs_1d.max() - fgs_1d.min())
        
        airs_1d = airs_1d
        fgs_1d = fgs_1d
    

        airs_frames = torch.tensor(airs_frames).float().unsqueeze(1)
        fgs_frames = torch.tensor(fgs_frames).float().unsqueeze(1)
        
       # --------------------------------------------------------------------------------------------- 
     
        airs_frames_2 = bin_obs(airs_ch0_processed, binning=2)
        fgs_frames_2  = bin_obs(fgs1_processed, binning=25)
        
        airs_1d_2     = torch.tensor(np.sum(airs_frames_2, axis=(1,2))).float()
        fgs_1d_2      = torch.tensor(np.sum(fgs_frames_2 , axis=(1,2))).float()
        
        airs_1d_2 = np.array(airs_1d_2)
        fgs_1d_2 = np.array(fgs_1d_2)
        
        input_vector = final_vector( airs_1d_2 , fgs_1d_2)
        
        dic = {
            'airs_frames': airs_frames,
            'fgs_frames': fgs_frames,
            'airs_1d': airs_1d,
            'fgs_1d': fgs_1d,
        }
        
        return planet, input_vector , dic

In [10]:
def smooth_signal(signal):
    x = np.arange(len(signal))
    smoothed = lowess(signal,x,frac=0.2,it=4,delta=0.1 * np.std(signal),return_sorted=False )
    return smoothed


In [9]:
torch.set_printoptions(precision=10)


In [21]:
device1 = torch.device('cuda')
device2 = torch.device("cpu")

airs_frames = 1125
fgs_frames = 1125

model1 = Model(airs_frames, fgs_frames).to(device1)
model1 = nn.DataParallel(model1).to(device1)

model2 = ResidualModel().to(device2)


weights1="/kaggle/input/arieldata/main_model_weights_3.pth"
weights2= "/kaggle/input/arieldata/res_model_weights_4.pth"


checkpoint1 = torch.load(weights1, map_location=device1 , weights_only=True)
model1.load_state_dict(checkpoint1['model_state_dict'])

checkpoint2 = torch.load(weights2, map_location=device2 , weights_only=True)
model2.load_state_dict(checkpoint2['model_state_dict'])


cuda_count = torch.cuda.device_count()
print(f"Number of CUDA devices: {cuda_count}")

Number of CUDA devices: 2


In [24]:
dataset = PlanetDataset(mode="test")
batchsize=1
t=1
dataloader = DataLoader(dataset, batch_size=batchsize, shuffle=False , num_workers=2)

model1.eval()
model2.eval()
final = []
i=0
vals=[]

with torch.no_grad():
    for planet , vec,  dic in dataloader:
        
#         print("processing" , planet )
        vals.append(planet.item())
        airs_frames = dic['airs_frames'].to(device1)
        fgs_frames  = dic['fgs_frames'].to(device1)
        airs_1d     = dic['airs_1d'].to(device1)
        fgs_1d      = dic['fgs_1d'].to(device1)
        
        output         = model1(airs_frames, fgs_frames, airs_1d, fgs_1d)
       
    # --------------------------------------------------------------------------------------------- 
        
        scales = model2(vec.float())  
        scales = np.array(scales.detach().cpu())
        out_1 = np.array(output.detach().cpu())
        
#         out_arr =     out_1[0] 
        out_arr =     out_1[0] / scales[0]
#         print(out_arr[:5])
    
        
#         out_arr = savgol_filter(out_arr, 50, 1)
        out_arr = smooth_signal(out_arr)

#         print(out_arr[:5])
        
        replacement_value = 0.0015
        out_arr = np.where(np.isnan(out_arr) | (out_arr == 0), replacement_value, out_arr)

        outs  = np.abs(out_arr)
        final.append(outs)
            
print("done")

done


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


In [27]:
print(len(final))
# print((out[:5]))


1


In [28]:
# print(out_arr[0] , planet)
# print(np.array(final).shape)
# print("yes")

In [29]:

predictions = np.array(final)
columns = ['planet_id'] + [f'wl_{i+1}' for i in range(283)] + [f'sigma_{i+1}' for i in range(283)]
submit_df = pd.DataFrame(columns=columns)

uncertainity = 7e-4
print(uncertainity)
rows_list = []  

for i in range(len(vals)):  
    row_data = {
        'planet_id': vals[i]
    }
    
    for j in range(283):
        row_data[f'wl_{j+1}'] = predictions[i, j]
    
    for j in range(283):
        row_data[f'sigma_{j+1}'] = uncertainity
    
    rows_list.append(row_data)

submit_df = pd.DataFrame(rows_list)

submit_df.to_csv("submission.csv", index=False)
print("saved")

0.0005
saved


In [12]:
# x=pd.read_csv("/kaggle/working/submission.csv")
# print(x)